# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 24 2022 10:01AM,249229,10,0086098851,ISDIN Corporation,Released
1,Oct 24 2022 10:01AM,249229,10,0086098852,ISDIN Corporation,Released
2,Oct 24 2022 10:01AM,249229,10,0086098857,ISDIN Corporation,Released
3,Oct 24 2022 10:01AM,249229,10,0086098854,ISDIN Corporation,Released
4,Oct 24 2022 10:01AM,249229,10,0086098855,ISDIN Corporation,Released
5,Oct 24 2022 10:01AM,249229,10,0086098856,ISDIN Corporation,Released
6,Oct 24 2022 10:01AM,249229,10,0086098858,ISDIN Corporation,Released
7,Oct 24 2022 10:01AM,249229,10,0086098859,ISDIN Corporation,Released
8,Oct 24 2022 10:01AM,249229,10,0086098861,ISDIN Corporation,Released
9,Oct 24 2022 10:01AM,249229,10,0086098860,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,249225,Released,53
14,249226,Released,5
15,249227,Executing,1
16,249228,Released,60
17,249229,Released,59


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249225,NaN,53.0
249226,NaN,5.0
249227,1.0,NaN
249228,NaN,60.0
249229,NaN,59.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249205,0.0,3.0
249209,13.0,6.0
249211,0.0,25.0
249212,0.0,10.0
249213,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249205,0,3
249209,13,6
249211,0,25
249212,0,10
249213,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249205,0,3
1,249209,13,6
2,249211,0,25
3,249212,0,10
4,249213,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249205,,3
1,249209,13,6
2,249211,,25
3,249212,,10
4,249213,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 24 2022 10:01AM,249229,10,ISDIN Corporation
59,Oct 24 2022 10:01AM,249228,10,ISDIN Corporation
119,Oct 24 2022 9:55AM,249225,10,ISDIN Corporation
172,Oct 24 2022 9:55AM,249224,10,ISDIN Corporation
234,Oct 24 2022 9:55AM,249219,10,ISDIN Corporation
291,Oct 24 2022 9:47AM,249226,19,Eywa Pharma Inc.
296,Oct 24 2022 9:43AM,249227,21,"NBTY Global, Inc."
297,Oct 24 2022 9:26AM,249223,10,"Snap Medical Industries, LLC"
304,Oct 24 2022 9:23AM,249221,10,Bio-PRF
305,Oct 24 2022 9:20AM,249220,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 24 2022 10:01AM,249229,10,ISDIN Corporation,,59
1,Oct 24 2022 10:01AM,249228,10,ISDIN Corporation,,60
2,Oct 24 2022 9:55AM,249225,10,ISDIN Corporation,,53
3,Oct 24 2022 9:55AM,249224,10,ISDIN Corporation,,62
4,Oct 24 2022 9:55AM,249219,10,ISDIN Corporation,,57
5,Oct 24 2022 9:47AM,249226,19,Eywa Pharma Inc.,,5
6,Oct 24 2022 9:43AM,249227,21,"NBTY Global, Inc.",1,
7,Oct 24 2022 9:26AM,249223,10,"Snap Medical Industries, LLC",,7
8,Oct 24 2022 9:23AM,249221,10,Bio-PRF,,1
9,Oct 24 2022 9:20AM,249220,10,Bio-PRF,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 10:01AM,249229,10,ISDIN Corporation,59,
1,Oct 24 2022 10:01AM,249228,10,ISDIN Corporation,60,
2,Oct 24 2022 9:55AM,249225,10,ISDIN Corporation,53,
3,Oct 24 2022 9:55AM,249224,10,ISDIN Corporation,62,
4,Oct 24 2022 9:55AM,249219,10,ISDIN Corporation,57,
5,Oct 24 2022 9:47AM,249226,19,Eywa Pharma Inc.,5,
6,Oct 24 2022 9:43AM,249227,21,"NBTY Global, Inc.",,1
7,Oct 24 2022 9:26AM,249223,10,"Snap Medical Industries, LLC",7,
8,Oct 24 2022 9:23AM,249221,10,Bio-PRF,1,
9,Oct 24 2022 9:20AM,249220,10,Bio-PRF,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 10:01AM,249229,10,ISDIN Corporation,59,
1,Oct 24 2022 10:01AM,249228,10,ISDIN Corporation,60,
2,Oct 24 2022 9:55AM,249225,10,ISDIN Corporation,53,
3,Oct 24 2022 9:55AM,249224,10,ISDIN Corporation,62,
4,Oct 24 2022 9:55AM,249219,10,ISDIN Corporation,57,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 10:01AM,249229,10,ISDIN Corporation,59.0,NaN
1,Oct 24 2022 10:01AM,249228,10,ISDIN Corporation,60.0,NaN
2,Oct 24 2022 9:55AM,249225,10,ISDIN Corporation,53.0,NaN
3,Oct 24 2022 9:55AM,249224,10,ISDIN Corporation,62.0,NaN
4,Oct 24 2022 9:55AM,249219,10,ISDIN Corporation,57.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 10:01AM,249229,10,ISDIN Corporation,59.0,0.0
1,Oct 24 2022 10:01AM,249228,10,ISDIN Corporation,60.0,0.0
2,Oct 24 2022 9:55AM,249225,10,ISDIN Corporation,53.0,0.0
3,Oct 24 2022 9:55AM,249224,10,ISDIN Corporation,62.0,0.0
4,Oct 24 2022 9:55AM,249219,10,ISDIN Corporation,57.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3738280,354.0,13.0
19,498431,8.0,0.0
21,249227,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3738280,354.0,13.0
1,19,498431,8.0,0.0
2,21,249227,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,354.0,13.0
1,19,8.0,0.0
2,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,354.0
1,19,Released,8.0
2,21,Released,0.0
3,10,Executing,13.0
4,19,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,21
Status,,,
Executing,13.0,0.0,1.0
Released,354.0,8.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,21
0,Executing,13.0,0.0,1.0
1,Released,354.0,8.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,21
0,Executing,13.0,0.0,1.0
1,Released,354.0,8.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()